안녕하세요.

IR팀에서 진행해온 SGA 업계 및 경쟁사 현황 자료를 시스템화 검토 진행 바랍니다.
이는 IR팀 입사 시 업계 파악 뿐 아니라,  주요 임직원들에게 공유 해 정보를 제공 하기 위함입니다.

기존에 수동으로 네이버 증권 정보에서 리서치 하여, 엑셀 자료에 반영 하였던 것으로
네이버 증권 정보 크롤링을 통해 작업 검토를 해보면 좋겠습니다.

추출되는 양식은 엑셀 외에도 좋으니, (파워BI / 원노트 등등)
자유로이 해당 항목이 적용 되도록 업무 진행 해 주시기 바랍니다.

경쟁사 : 실적 현황표에 기재된 회사들로 업데이트 (보안 / IT서비스)  
시가총액 / 종가 / 등락율 / 등락폭 / 공시 (전체) / 뉴스 (뉴스는 타이틀 위주로 1건)  
업계 : 경쟁사 자료 세팅 후 추후 진행 예정 하도록 하겠습니다.
업무 진행하면서 궁금한 사항은 문의 주세요.



In [ ]:
import pandas as pd

In [56]:
company_dic = {"SGA": "049470", "SGA솔루션즈":"184230", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
               "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
               "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
               "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
               "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680", "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250"
               }



# company_dic = {
#     'SGA솔루션즈': '184230',
#     '시큐브': '131090',
#     '안랩': '053800',
#     '모니터랩': '434480',
#     '이스트소프트': '047560',
#     '윈스': '136540',
#     '한국정보인증': '053300',
#     '라온시큐어': '042510',
#     '파수': '150900',
#     '지란지교시큐리티': '208350',
#     '지니언스': '263860',
#     '한컴위드': '054920',
#     '드림시큐리티': '203650',
#     '케이사인': '192250',
#     "샌즈랩":"411080",
#     '이글루': '067920',
#     '파이오링크': '170790',
#     '시큐센': '232830',
#     '시큐레터': '418250',
#     'SGA': '049470',
#     '한글과컴퓨터': '030520',
#     'MDS테크': '086960',
#     '아이티센': '124500',
#     '쌍용정보통신': '010280'
# }

# 보이스아이, 이글루시큐리티, 코닉글로리,한컴MDS 없음

com_keys = list(company_dic.keys())

### https://finance.naver.com/item/sise.naver?code=049470

In [28]:
import requests as req
!pip install bs4
from bs4 import BeautifulSoup as bs


# 시총 + 뉴스

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for company_name, stock_code in company_dic.items():
    # 종목의 일별 시세 URL
    sise_day_url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    # 종목의 상세 시세 URL
    sise_url = f"https://finance.naver.com/item/sise.naver?code={stock_code}"
    # 종목의 뉴스 URL
    news_url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={company_name}"

    # 최신 데이터 가져오기 (일별 시세)
    response = requests.get(sise_day_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 테이블 선택
    table = soup.find("table", class_="type2")
    rows = table.find_all("tr")

    # 첫 번째 유효한 데이터 추출
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:  # 데이터가 있는 행만 처리
            date = cols[0].get_text(strip=True)
            close_price = cols[1].get_text(strip=True)
            diff = cols[2].get_text(strip=True)
            open_price = cols[3].get_text(strip=True)
            high_price = cols[4].get_text(strip=True)
            low_price = cols[5].get_text(strip=True)
            volume = cols[6].get_text(strip=True)
            break  # 첫 번째 유효한 행만 처리

    # 등락률 가져오기
    response = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    rate_tag = soup.select_one("strong#_rate > span")
    rate = rate_tag.text.strip() if rate_tag else "N/A"

    # 시가총액 가져오기
    market_cap_tag = soup.select_one("em#_market_sum")
    market_cap = market_cap_tag.text.strip() + "억 원" if market_cap_tag else "N/A"

    # 뉴스 URL 가져오기
    response = requests.get(news_url, headers=headers)
    bs_sga = BeautifulSoup(response.text, 'lxml')
    link = bs_sga.select_one('a.news_tit')  # 'a.news_tit' 선택자 사용
    link = link['href'] if link else "N/A"

    # 데이터 리스트에 추가
    all_data.append({
        "회사": company_name,
        "날짜": date,
        "시가총액": market_cap,  # 시가총액 추가
        "종가": close_price,
        "전일대비": diff,
        "등락률": rate,
        "시가": open_price,
        "고가": high_price,
        "저가": low_price,
        "거래량": volume,
        "뉴스": link
    })

# DataFrame 생성
df = pd.DataFrame(all_data)
# jasa = df.iloc[:2]
# secur = df.iloc[2:17]
# its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# voiceye_row = {
#     "회사": "보이스아이",
#     "날짜": date,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# df = pd.concat([pd.DataFrame([new_row]), jasa, pd.DataFrame([voiceye_row]), pd.DataFrame([sec_row]), secur, pd.DataFrame([it_row]), its], ignore_index=True)

# CSV 저장
df.to_csv("latest_stock_prices_with_market_cap.csv", index=False, encoding="utf-8-sig")
df.to_excel("latest_stock_prices_with_market_cap.xlsx", index=False, engine="openpyxl")


In [ ]:
df

,회사,날짜,시가총액,종가,전일대비,등락률,시가,고가,저가,거래량,뉴스
0,SGA솔루션즈,2025.01.10,314억 원,502,하락20,-3.83%,492,506,492,"585,412",http://www.boannews.com/media/view.asp?idx=135...
1,시큐브,2025.01.10,400억 원,940,상승20,+2.17%,911,975,890,"229,484",https://www.pinpointnews.co.kr/news/articleVie...
2,안랩,2025.01.10,"7,789억 원","70,000",하락400,-0.57%,"70,000","71,200","69,500","76,179",https://www.yna.co.kr/view/AKR2025011004240001...
3,모니터랩,2025.01.10,528억 원,"4,310",하락170,-3.79%,"4,425","4,475","4,310","289,547",https://news.mtn.co.kr/news-detail/20250109063...
4,이스트소프트,2025.01.10,"2,693억 원","23,200",하락450,-1.90%,"23,550","23,750","22,900","293,662",https://biz.chosun.com/it-science/ict/2025/01/...
5,윈스,2025.01.10,"1,389억 원","11,310",하락940,-7.67%,"12,330","12,330","11,150","106,024",https://www.etoday.co.kr/news/view/2432892
6,한국정보인증,2025.01.10,"1,912억 원","4,505",상승75,+1.69%,"4,425","4,560","4,425","564,284",https://economist.co.kr/article/view/ecn202501...
7,라온시큐어,2025.01.10,"1,143억 원","2,040",상승5,+0.25%,"2,040","2,055","1,980","294,689",http://jmagazine.joins.com/forbes/view/340666
8,파수,2025.01.10,576억 원,"4,930",하락60,-1.20%,"4,975","4,980","4,890","12,420",http://www.gndomin.com/news/articleView.html?i...
9,지란지교시큐리티,2025.01.10,260억 원,"2,940",상승5,+0.17%,"2,935","2,960","2,920","3,641",https://www.newsis.com/view/NISX20250110_00030...


## 지정된 엑셀 형식에 데이터 집어넣기

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# 1. 서식화된 엑셀 파일 열기
formatted_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황.xlsx"  # 서식화된 엑셀 파일 경로
output_file = "latest_stock_prices_with_market_cap.xlsx"  # 저장될 파일 경로

# 2. 기존 서식화된 엑셀 파일 불러오기
wb = load_workbook(formatted_file)
sheet = wb["경쟁사주가"]  # 데이터 삽입할 시트 이름

# 3. 데이터 추가 위치 설정 (예: A5 셀부터 시작)
start_row = 5  # 데이터 시작 행
start_col = 1  # 데이터 시작 열 (1열 = 'A')

# 4. 데이터프레임에서 데이터 삽입
for row_index, row_data in enumerate(dataframe_to_rows(df, index=False, header=False), start=start_row):
    for col_index, value in enumerate(row_data, start=start_col):
        sheet.cell(row=row_index, column=col_index, value=value)

# 5. 엑셀 파일 저장
wb.save(output_file)
print(f"서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: {output_file}")


서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: latest_stock_prices_with_market_cap.xlsx


In [ ]:
from openpyxl import load_workbook

# 서식화된 엑셀 파일 열기
formatted_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황.xlsx"
wb = load_workbook(formatted_file)

# 엑셀 파일에 있는 모든 시트 이름 출력
print(wb.sheetnames)


['작업용2', '경쟁사주가', '시트1', '업계현황']


# Pract

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for company_name, stock_code in company_dic.items():
    # 종목의 일별 시세 URL
    sise_day_url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    # 종목의 상세 시세 URL
    sise_url = f"https://finance.naver.com/item/sise.naver?code={stock_code}"
    # 종목의 뉴스 URL
    news_url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={company_name}"

    # 최신 데이터 가져오기 (일별 시세)
    response = requests.get(sise_day_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 테이블 선택
    table = soup.find("table", class_="type2")
    rows = table.find_all("tr")

    # 첫 번째 유효한 데이터 추출
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:  # 데이터가 있는 행만 처리
            date = cols[0].get_text(strip=True)
            close_price = cols[1].get_text(strip=True)
            diff = cols[2].get_text(strip=True)
            diff = diff[2:]
            open_price = cols[3].get_text(strip=True)
            high_price = cols[4].get_text(strip=True)
            low_price = cols[5].get_text(strip=True)
            volume = cols[6].get_text(strip=True)
            break  # 첫 번째 유효한 행만 처리


    # 등락률 가져오기
    response = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    rate_tag = soup.select_one("strong#_rate > span")
    rate = rate_tag.text.strip() if rate_tag else "N/A"
    if rate[0] == "+": rate = rate[1:]


    # 현재가 가져오기
    response2 = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response2.text, "html.parser")
    current_tag = soup.select_one("#_nowVal")
    current_price = current_tag.text.strip() if current_tag else "N/A"


    # 거래대금 가져오기
    response3 = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response3.text, "html.parser")
    trade_mon_tag = soup.select_one("#_amount")
    trade_mon = trade_mon_tag.text.strip() if trade_mon_tag else "N/A"

    # 시가총액 가져오기
    market_cap_tag = soup.select_one("em#_market_sum")
    market_cap = market_cap_tag.text.strip() if market_cap_tag else "N/A"

    # 뉴스 URL 가져오기
    response = requests.get(news_url, headers=headers)
    bs_sga = BeautifulSoup(response.text, 'lxml')
    link = bs_sga.select_one('a.news_tit')  # 'a.news_tit' 선택자 사용
    link = link['href'] if link else "N/A"

    # 데이터 리스트에 추가
    all_data.append({
        "회사": company_name,
        # "현재가":current_price,
        "종가": close_price,
        "전일대비": diff,
        "등락률": rate,
        "시가": open_price,
        "고가": high_price,
        "저가": low_price,
        "거래량": volume,
        "거래대금":trade_mon,
        "시가총액(억)": market_cap,  # 시가총액 추가
        "날짜": date
    })

# DataFrame 생성
df = pd.DataFrame(all_data)
jasa = df.iloc[:2]
elsee = df.iloc[2:]
# its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

voiceye_row = {
    "회사": "SGA임베디드*(코스닥 종목)",
    "종가": "2965",
    "전일대비": "0",
    "등락률":"0%",
    "시가": "2965",
    "고가": "2965",
    "저가": "2965",
    "거래량": "0",
    "거래대금": "0",
    "시가총액(억)": "161",
    "날짜": "-"

}

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

df = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)

# CSV 저장
df.to_csv("latest_stock_prices_with_market_cap.csv", index=False, encoding="utf-8-sig")
df.to_excel("latest_stock_prices_with_market_cap.xlsx", index=False, engine="openpyxl")

In [ ]:
df

,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금,시가총액(억),날짜
0,SGA,327,7,-2.10%,335,358,323,"960,330",327,192,2025.01.13
1,SGA솔루션즈,485,17,-3.39%,502,502,485,"125,024",62,304,2025.01.13
2,SGA임베디드*(코스닥 종목),2965,0,0%,2965,2965,2965,0,0,161,-
3,안랩,"69,400",500,-0.72%,"69,800","70,500","65,400","21,995","1,519","7,722",2025.01.13
4,시큐브,934,11,-1.16%,945,946,918,"36,850",34,398,2025.01.13
5,윈스,"11,630",210,1.84%,"11,540","11,670","11,490","15,892",184,"1,428",2025.01.13
6,이글루,"5,250",50,-0.94%,"5,240","5,290","5,230","5,757",30,577,2025.01.13
7,한컴위드,"2,990",85,2.93%,"2,955","3,180","2,955","648,906","2,012",848,2025.01.13
8,네오리진,"1,056",10,0.96%,"1,046","1,056","1,040","8,792",9,226,2025.01.13
9,케이사인,"8,060",70,-0.86%,"8,040","8,060","8,010","1,629",13,570,2025.01.13


In [ ]:
dfp = df.copy()

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

# 데이터프레임 전처리 함수
def preprocess_dataframe(df):
    for column in df.columns:
        if (column == "회사") or (column == "날짜"):  # 회사 이름은 변환하지 않음
            continue
        try:
            # 퍼센트 처리: '%'가 포함된 경우
            if df[column].astype(str).str.contains('%').any():
                continue

            else:
                # 숫자 변환
                df[column] = df[column].str.replace(',', '', regex=False).astype(float)
        except Exception as e:
            print(f"Error processing column {column}: {e}")
    return df


# 데이터프레임 전처리
dfp = preprocess_dataframe(dfp)
display(dfp)

# 1. 서식화된 엑셀 파일 열기
formatted_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황2.xlsx"  # 서식화된 엑셀 파일 경로
output_file = "latest_stock_prices_with_market_cap.xlsx"  # 저장될 파일 경로

# 2. 기존 서식화된 엑셀 파일 불러오기
wb = load_workbook(formatted_file)
sheet = wb["작업용2"]  # 데이터 삽입할 시트 이름

# 3. 데이터 추가 위치 설정 (예: A3 셀부터 시작)
start_row = 2  # 데이터 시작 행
start_col = 2  # 데이터 시작 열 (1열 = 'A')

# 4. 데이터프레임에서 데이터 삽입
for row_index, row_data in enumerate(dataframe_to_rows(dfp, index=False, header=True), start=start_row):
    for col_index, value in enumerate(row_data, start=start_col):
        cell = sheet.cell(row=row_index, column=col_index, value=value)

        # 숫자 데이터 처리
        if isinstance(value, (int, float)):
            cell.number_format = numbers.FORMAT_NUMBER  # 일반 숫자 형식

        # 퍼센트 데이터 처리
        elif isinstance(value, str) and "%" in value:
            try:
                # 퍼센트 값을 숫자로 변환 (예: "25%" -> 0.25)
                percent_value = float(value.strip('%'))/100
                cell.value = percent_value
                cell.number_format = numbers.FORMAT_PERCENTAGE  # 퍼센트 형식
            except ValueError:
                pass  # 변환 실패 시 문자열 그대로 저장

# 5. 엑셀 파일 저장
wb.save(output_file)
print(f"서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: {output_file}")


,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금,시가총액(억),날짜
0,SGA,327.0,7.0,-2.10%,335.0,358.0,323.0,960330.0,327.0,192.0,2025.01.13
1,SGA솔루션즈,485.0,17.0,-3.39%,502.0,502.0,485.0,125024.0,62.0,304.0,2025.01.13
2,SGA임베디드*(코스닥 종목),2965.0,0.0,0%,2965.0,2965.0,2965.0,0.0,0.0,161.0,-
3,안랩,69400.0,500.0,-0.72%,69800.0,70500.0,65400.0,21995.0,1519.0,7722.0,2025.01.13
4,시큐브,934.0,11.0,-1.16%,945.0,946.0,918.0,36850.0,34.0,398.0,2025.01.13
5,윈스,11630.0,210.0,1.84%,11540.0,11670.0,11490.0,15892.0,184.0,1428.0,2025.01.13
6,이글루,5250.0,50.0,-0.94%,5240.0,5290.0,5230.0,5757.0,30.0,577.0,2025.01.13
7,한컴위드,2990.0,85.0,2.93%,2955.0,3180.0,2955.0,648906.0,2012.0,848.0,2025.01.13
8,네오리진,1056.0,10.0,0.96%,1046.0,1056.0,1040.0,8792.0,9.0,226.0,2025.01.13
9,케이사인,8060.0,70.0,-0.86%,8040.0,8060.0,8010.0,1629.0,13.0,570.0,2025.01.13


서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: latest_stock_prices_with_market_cap.xlsx


# news crawling - selenium

In [24]:
import requests
from bs4 import BeautifulSoup

# 대상 URL
url = "https://finance.naver.com/item/news.naver?code=049470"

# HTTP 요청
response = requests.get(url)
response.encoding = 'utf-8'  # 한글 깨짐 방지

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# 첫 번째 뉴스 기사 URL 추출

news = soup.select('body > div > table.type5 > tbody > tr.first.relation_tit > td.title > a')  # 첫 번째 뉴스 링크
print(news)

[]


In [115]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for company_name in company_dic.keys():
  url = f"https://search.naver.com/search.naver?where=news&query={company_name}&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=3&office_section_code=&news_office_checked=&nso=&is_sug_officeid=0&office_category=3&service_area=0"



  # 최신 데이터 가져오기 (일별 시세)
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  first_news = soup.select_one('div.news_area a.news_tit')
  if first_news:
  # 기사 제목과 URL 추출
    news_title = first_news.get_text()
    news_url = first_news['href']
    print(f'기사 제목: {news_title}')
    print(f'기사 URL: {news_url}')
  else:
      print('뉴스 기사를 찾을 수 없습니다.')



    # 데이터 리스트에 추가
  all_data.append({

      # "현재가":current_price,
      "제목": news_title,
      "url": news_url

    })

# DataFrame 생성
df = pd.DataFrame(all_data)
jasa = df.iloc[:2]
elsee = df.iloc[2:]
# # its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

voiceye_row = {

     "제목": "보이스아이, 시각장애인 대상 코로나 19 정보전달을 위한 ‘음성변환 바코드’ 무상 제공",
     "url": "https://www.etoday.co.kr/news/view/1870936",


 }

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

df = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)

# CSV 저장
df.to_csv("latest_news_of_companies.csv", index=False, encoding="utf-8-sig")
df.to_excel("latest_news_of_companies.xlsx", index=False, engine="openpyxl")

기사 제목: SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...
기사 URL: http://www.boannews.com/media/view.asp?idx=135516&kind=6
기사 제목: SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...
기사 URL: http://www.boannews.com/media/view.asp?idx=135516&kind=6
기사 제목: 안랩, '2025년 데이터바우처 지원사업' 공급기업으로 선정
기사 URL: http://www.enewstoday.co.kr/news/articleView.html?idxno=2224401
기사 제목: 시큐브 주가 콧노래...통합계정권한관리 솔루션 iGRIFFIN 3.0m 출시 호평
기사 URL: https://www.pinpointnews.co.kr/news/articleView.html?idxno=311345
기사 제목: [오늘의 주요공시] 윈스·다원시스·케이엔제이 등
기사 URL: https://www.etoday.co.kr/news/view/2432892
기사 제목: 이글루코퍼레이션, 과기부·KISA ‘AI 보안 사업화 우수기업’ 선정
기사 URL: https://biz.chosun.com/it-science/ict/2025/01/09/XY6CJ776TBAABBMILFXBA7D62Q/?utm_source=naver&utm_medium=original&utm_campaign=biz
기사 제목: 한컴위드, 국내 최초 양자내성암호 포함 암호모듈 국정원 KCMVP 검증
기사 URL: https://zdnet.co.kr/view/?no=20250113105118
기사 제목: 네오리진, 수집형 모바일 RPG '에이펙스 걸스' 그랜드 오픈
기사 URL: http://news.mt.co.kr/mtview.php?no=2025010908475418070
기사 제목: 케이사인, 공공 데이터 효율적 활용 체계 구축
기사 URL: https://www.dat

In [117]:
df2 = df.copy()

In [119]:
df2

,제목,url
0,"SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...",http://www.boannews.com/media/view.asp?idx=135...
1,"SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...",http://www.boannews.com/media/view.asp?idx=135...
2,"보이스아이, 시각장애인 대상 코로나 19 정보전달을 위한 ‘음성변환 바코드’ 무상 제공",https://www.etoday.co.kr/news/view/1870936
3,"안랩, '2025년 데이터바우처 지원사업' 공급기업으로 선정",http://www.enewstoday.co.kr/news/articleView.h...
4,시큐브 주가 콧노래...통합계정권한관리 솔루션 iGRIFFIN 3.0m 출시 호평,https://www.pinpointnews.co.kr/news/articleVie...
5,[오늘의 주요공시] 윈스·다원시스·케이엔제이 등,https://www.etoday.co.kr/news/view/2432892
6,"이글루코퍼레이션, 과기부·KISA ‘AI 보안 사업화 우수기업’ 선정",https://biz.chosun.com/it-science/ict/2025/01/...
7,"한컴위드, 국내 최초 양자내성암호 포함 암호모듈 국정원 KCMVP 검증",https://zdnet.co.kr/view/?no=20250113105118
8,"네오리진, 수집형 모바일 RPG '에이펙스 걸스' 그랜드 오픈",http://news.mt.co.kr/mtview.php?no=20250109084...
9,"케이사인, 공공 데이터 효율적 활용 체계 구축",https://www.datanet.co.kr/news/articleView.htm...


In [120]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황2.xlsx"
output_path = "/content/뉴스_데이터.xlsx"



# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "작업용2" in wb.sheetnames:
    ws = wb["작업용2"]
else:
    ws = wb.create_sheet("작업용2")

# 데이터 추가 시작 셀
start_row = 3  # 행 번호
start_col = 14  # N 열 (엑셀에서 N은 14번째 열)

# 데이터프레임의 열 제목 추가
for col_idx, column_name in enumerate(df2.columns, start=start_col):
    ws.cell(row=1, column=col_idx, value=column_name)

# 데이터 추가
for row_idx, row in df2.iterrows():
    for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
        cell = ws.cell(row=row_idx + start_row, column=col_idx, value=value)
        # URL 컬럼의 경우 하이퍼링크 설정
        if col_name == "url":
            ws.cell(row=row_idx + start_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
            ws.cell(row=row_idx + start_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")

데이터프레임이 /content/뉴스_데이터.xlsx 파일에 성공적으로 추가되었습니다.


In [97]:
df

,제목,url
0,"SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...",http://www.boannews.com/media/view.asp?idx=135...
1,"SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...",http://www.boannews.com/media/view.asp?idx=135...
2,"보이스아이, 시각장애인 대상 코로나 19 정보전달을 위한 ‘음성변환 바코드’ 무상 제공",https://www.etoday.co.kr/news/view/1870936
3,"안랩, '2025년 데이터바우처 지원사업' 공급기업으로 선정",http://www.enewstoday.co.kr/news/articleView.h...
4,시큐브 주가 콧노래...통합계정권한관리 솔루션 iGRIFFIN 3.0m 출시 호평,https://www.pinpointnews.co.kr/news/articleVie...
5,[오늘의 주요공시] 윈스·다원시스·케이엔제이 등,https://www.etoday.co.kr/news/view/2432892
6,"이글루코퍼레이션, 과기부·KISA ‘AI 보안 사업화 우수기업’ 선정",https://biz.chosun.com/it-science/ict/2025/01/...
7,"한컴위드, 국내 최초 양자내성암호 포함 암호모듈 국정원 KCMVP 검증",https://zdnet.co.kr/view/?no=20250113105118
8,"네오리진, 수집형 모바일 RPG '에이펙스 걸스' 그랜드 오픈",http://news.mt.co.kr/mtview.php?no=20250109084...
9,"케이사인, 공공 데이터 효율적 활용 체계 구축",https://www.datanet.co.kr/news/articleView.htm...


# total

## 뉴스 제외한 수치 크롤링하기

In [131]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for company_name, stock_code in company_dic.items():
    # 종목의 일별 시세 URL
    sise_day_url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    # 종목의 상세 시세 URL
    sise_url = f"https://finance.naver.com/item/sise.naver?code={stock_code}"
    # 종목의 뉴스 URL
    news_url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={company_name}"

    # 최신 데이터 가져오기 (일별 시세)
    response = requests.get(sise_day_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 테이블 선택
    table = soup.find("table", class_="type2")
    rows = table.find_all("tr")

    # 첫 번째 유효한 데이터 추출
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:  # 데이터가 있는 행만 처리
            date = cols[0].get_text(strip=True)
            close_price = cols[1].get_text(strip=True)
            diff = cols[2].get_text(strip=True)
            diff = diff[2:]
            open_price = cols[3].get_text(strip=True)
            high_price = cols[4].get_text(strip=True)
            low_price = cols[5].get_text(strip=True)
            volume = cols[6].get_text(strip=True)
            break  # 첫 번째 유효한 행만 처리


    # 등락률 가져오기
    response = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    rate_tag = soup.select_one("strong#_rate > span")
    rate = rate_tag.text.strip() if rate_tag else "N/A"
    if rate[0] == "+": rate = rate[1:]


    # 현재가 가져오기
    response2 = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response2.text, "html.parser")
    current_tag = soup.select_one("#_nowVal")
    current_price = current_tag.text.strip() if current_tag else "N/A"


    # 거래대금 가져오기
    response3 = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response3.text, "html.parser")
    trade_mon_tag = soup.select_one("#_amount")
    trade_mon = trade_mon_tag.text.strip() if trade_mon_tag else "N/A"

    # 시가총액 가져오기
    market_cap_tag = soup.select_one("em#_market_sum")
    market_cap = market_cap_tag.text.strip() if market_cap_tag else "N/A"

    # 뉴스 URL 가져오기
    response = requests.get(news_url, headers=headers)
    bs_sga = BeautifulSoup(response.text, 'lxml')
    link = bs_sga.select_one('a.news_tit')  # 'a.news_tit' 선택자 사용
    link = link['href'] if link else "N/A"

    # 데이터 리스트에 추가
    all_data.append({
        "회사": company_name,
        # "현재가":current_price,
        "종가": close_price,
        "전일대비": diff,
        "등락률": rate,
        "시가": open_price,
        "고가": high_price,
        "저가": low_price,
        "거래량": volume,
        "거래대금":trade_mon,
        "시가총액(억)": market_cap,  # 시가총액 추가
        "날짜": date
    })

# DataFrame 생성
df = pd.DataFrame(all_data)
jasa = df.iloc[:2]
elsee = df.iloc[2:]
# its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

voiceye_row = {
    "회사": "SGA임베디드*(코스닥 종목)",
    "종가": "2965",
    "전일대비": "0",
    "등락률":"0%",
    "시가": "2965",
    "고가": "2965",
    "저가": "2965",
    "거래량": "0",
    "거래대금": "0",
    "시가총액(억)": "161",
    "날짜": "-"

}

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

df = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)

# CSV 저장
df.to_csv("latest_stock_prices_with_market_cap.csv", index=False, encoding="utf-8-sig")
df.to_excel("latest_stock_prices_with_market_cap.xlsx", index=False, engine="openpyxl")

In [132]:
df1 = df.copy()

## 뉴스 데이터 크롤링하기

In [133]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for company_name in company_dic.keys():
  url = f"https://search.naver.com/search.naver?where=news&query={company_name}&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=3&office_section_code=&news_office_checked=&nso=&is_sug_officeid=0&office_category=3&service_area=0"



  # 최신 데이터 가져오기 (일별 시세)
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  first_news = soup.select_one('div.news_area a.news_tit')
  if first_news:
  # 기사 제목과 URL 추출
    news_title = first_news.get_text()
    news_url = first_news['href']
    print(f'기사 제목: {news_title}')
    print(f'기사 URL: {news_url}')
  else:
      print('뉴스 기사를 찾을 수 없습니다.')



    # 데이터 리스트에 추가
  all_data.append({

      # "현재가":current_price,
      "제목": news_title,
      "url": news_url

    })

# DataFrame 생성
df2 = pd.DataFrame(all_data)
jasa = df2.iloc[:2]
elsee = df2.iloc[2:]
# # its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

voiceye_row = {

     "제목": "보이스아이, 시각장애인 대상 코로나 19 정보전달을 위한 ‘음성변환 바코드’ 무상 제공",
     "url": "https://www.etoday.co.kr/news/view/1870936",


 }

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

df2 = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)

# CSV 저장
df2.to_csv("latest_news_of_companies.csv", index=False, encoding="utf-8-sig")
df2.to_excel("latest_news_of_companies.xlsx", index=False, engine="openpyxl")

기사 제목: SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...
기사 URL: http://www.boannews.com/media/view.asp?idx=135516&kind=6
기사 제목: SGA솔루션즈 최영철 대표, “제로트러스트 풀스택 기업으로 시너지 낼 ...
기사 URL: http://www.boannews.com/media/view.asp?idx=135516&kind=6
기사 제목: 안랩, '2025년 데이터바우처 지원사업' 공급기업으로 선정
기사 URL: http://www.enewstoday.co.kr/news/articleView.html?idxno=2224401
기사 제목: 시큐브 주가 콧노래...통합계정권한관리 솔루션 iGRIFFIN 3.0m 출시 호평
기사 URL: https://www.pinpointnews.co.kr/news/articleView.html?idxno=311345
기사 제목: [오늘의 주요공시] 윈스·다원시스·케이엔제이 등
기사 URL: https://www.etoday.co.kr/news/view/2432892
기사 제목: 이글루코퍼레이션, 과기부·KISA ‘AI 보안 사업화 우수기업’ 선정
기사 URL: https://biz.chosun.com/it-science/ict/2025/01/09/XY6CJ776TBAABBMILFXBA7D62Q/?utm_source=naver&utm_medium=original&utm_campaign=biz
기사 제목: 한컴위드, 국내 최초 양자내성암호 포함 암호모듈 국정원 KCMVP 검증
기사 URL: https://zdnet.co.kr/view/?no=20250113105118
기사 제목: 네오리진, 수집형 모바일 RPG '에이펙스 걸스' 그랜드 오픈
기사 URL: http://news.mt.co.kr/mtview.php?no=2025010908475418070
기사 제목: 케이사인, 공공 데이터 효율적 활용 체계 구축
기사 URL: https://www.dat

# 엑셀화하기

In [134]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

# 데이터프레임 전처리 함수
def preprocess_dataframe(df):
    for column in df.columns:
        if (column == "회사") or (column == "날짜"):  # 회사 이름은 변환하지 않음
            continue
        try:
            # 퍼센트 처리: '%'가 포함된 경우
            if df[column].astype(str).str.contains('%').any():
                continue

            else:
                # 숫자 변환
                df[column] = df[column].str.replace(',', '', regex=False).astype(float)
        except Exception as e:
            print(f"Error processing column {column}: {e}")
    return df


# 데이터프레임 전처리
df1 = preprocess_dataframe(df1)
display(df1)

# 1. 서식화된 엑셀 파일 열기
formatted_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황2.xlsx"  # 서식화된 엑셀 파일 경로
output_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"  # 저장될 파일 경로

# 2. 기존 서식화된 엑셀 파일 불러오기
wb = load_workbook(formatted_file)
sheet = wb["작업용2"]  # 데이터 삽입할 시트 이름

# 3. 데이터 추가 위치 설정 (예: A3 셀부터 시작)
start_row = 2  # 데이터 시작 행
start_col = 2  # 데이터 시작 열 (1열 = 'A')

# 4. 데이터프레임에서 데이터 삽입
for row_index, row_data in enumerate(dataframe_to_rows(df1, index=False, header=True), start=start_row):
    for col_index, value in enumerate(row_data, start=start_col):
        cell = sheet.cell(row=row_index, column=col_index, value=value)

        # 숫자 데이터 처리
        if isinstance(value, (int, float)):
            cell.number_format = numbers.FORMAT_NUMBER  # 일반 숫자 형식

        # 퍼센트 데이터 처리
        elif isinstance(value, str) and "%" in value:
            try:
                # 퍼센트 값을 숫자로 변환 (예: "25%" -> 0.25)
                percent_value = float(value.strip('%'))/100
                cell.value = percent_value
                cell.number_format = numbers.FORMAT_PERCENTAGE  # 퍼센트 형식
            except ValueError:
                pass  # 변환 실패 시 문자열 그대로 저장


# 5. 엑셀 파일 저장
wb.save(output_file)
print(f"서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: {output_file}")


,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금,시가총액(억),날짜
0,SGA,320.0,14.0,-4.19%,335.0,358.0,320.0,1406776.0,472.0,188.0,2025.01.13
1,SGA솔루션즈,486.0,16.0,-3.19%,502.0,502.0,482.0,224052.0,110.0,304.0,2025.01.13
2,SGA임베디드*(코스닥 종목),2965.0,0.0,0%,2965.0,2965.0,2965.0,0.0,0.0,161.0,-
3,안랩,69600.0,300.0,-0.43%,69800.0,70500.0,65400.0,45434.0,3146.0,7744.0,2025.01.13
4,시큐브,925.0,20.0,-2.12%,945.0,947.0,913.0,154040.0,143.0,394.0,2025.01.13
5,윈스,11440.0,20.0,0.18%,11540.0,11670.0,11440.0,31826.0,367.0,1405.0,2025.01.13
6,이글루,5270.0,30.0,-0.57%,5240.0,5300.0,5230.0,10525.0,55.0,579.0,2025.01.13
7,한컴위드,3030.0,125.0,4.30%,2955.0,3180.0,2955.0,851230.0,2601.0,855.0,2025.01.13
8,네오리진,1032.0,14.0,-1.34%,1046.0,1056.0,1022.0,21759.0,23.0,221.0,2025.01.13
9,케이사인,8110.0,20.0,-0.25%,8040.0,8110.0,7940.0,11137.0,89.0,573.0,2025.01.13


서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: /content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx


## 뉴스 데이터 엑셀화(수치+뉴스)

In [135]:
df = df2

In [136]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"
output_path = "/content/수치+뉴스데이터.xlsx"



# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 행 번호
start_col = 10  # N 열 (엑셀에서 N은 14번째 열)

# 데이터프레임의 열 제목 추가
for col_idx, column_name in enumerate(df.columns, start=start_col):
    ws.cell(row=1, column=col_idx, value=column_name)

# 데이터 추가
for row_idx, row in df.iterrows():
    for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
        cell = ws.cell(row=row_idx + start_row, column=col_idx, value=value)
        # URL 컬럼의 경우 하이퍼링크 설정
        if col_name == "url":
            ws.cell(row=row_idx + start_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
            ws.cell(row=row_idx + start_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")

데이터프레임이 /content/수치+뉴스데이터.xlsx 파일에 성공적으로 추가되었습니다.


In [138]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"
output_path = "/content/수치+뉴스데이터.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 10  # 데이터 시작 열 번호 (엑셀에서 N열은 14번째 열)

# 데이터프레임의 열 제목 추가
for col_idx, column_name in enumerate(df.columns, start=start_col):
    ws.cell(row=1, column=col_idx, value=column_name)

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가
for row_idx, row in df.iterrows():
    # 행 번호 계산 (엑셀의 row 번호와 매칭)
    excel_row_number = current_row

    # 7 또는 23일 경우 다음 행에 데이터를 밀어넣음
    if excel_row_number in [7, 23]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동

    for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
        cell = ws.cell(row=current_row, column=col_idx, value=value)
        # URL 컬럼의 경우 하이퍼링크 설정
        if col_name == "url":
            ws.cell(row=current_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
            ws.cell(row=current_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")



데이터프레임이 /content/수치+뉴스데이터.xlsx 파일에 성공적으로 추가되었습니다.
